<div align="center">
    <img src="https://www.sharif.ir/documents/20124/0/logo-fa-IR.png/4d9b72bc-494b-ed5a-d3bb-e7dfd319aec8?t=1609608338755" alt="Logo" width="200">
    <p><b> Reinforcement Learning Course, Dr. Rohban</b></p>
</div>


> - Full Name: Taha Majlesi
> - Student ID: 810101504

# Random Network Distillation (RND) with PPO - Homework Project

  

---

  


## 1. Introduction: Random Network Distillation (RND)

A common way of doing exploration is to visit states with a large prediction error of some quantity, for instance, the TD error or even random functions.  
The RND algorithm ([Exploration by Random Network Distillation](https://arxiv.org/abs/1810.12894)) aims at encouraging exploration by asking the exploration policy to more frequently undertake transitions where the prediction error of a random neural network function is high.

Formally, let $f^*_\theta(s')$ be a randomly chosen vector-valued function represented by a neural network.  
RND trains another neural network, $\hat{f}_\phi(s')$, to match the predictions of $f^*_\theta(s')$ under the distribution of datapoints in the buffer, as shown below:

$$
\phi^* = \arg\min_\phi \mathbb{E}_{s,a,s'\sim\mathcal{D}} \left[ \left\| \hat{f}_\phi(s') - f^*_\theta(s') \right\| \right]
$$

If a transition $(s, a, s')$ is in the distribution of the data buffer, the prediction error $\mathcal{E}_\phi(s')$ is expected to be small.  
On the other hand, for all unseen state-action tuples, it is expected to be large.

In practice, RND uses two critics:
- an exploitation critic $Q_R(s,a)$, which estimates returns based on the true rewards,
- and an exploration critic $Q_E(s,a)$, which estimates returns based on the exploration bonuses.

To stabilize training, prediction errors are normalized before being used.

---

## 2. What You Will Implement

  

You will implement the missing core components of Random Network Distillation (RND) combined with a Proximal Policy Optimization (PPO) agent inside the MiniGrid environment.

  

Specifically, you will:

  

- Complete the architecture of TargetModel and PredictorModel.

  

- Complete the initialization of weights for these models.

  

- Implement the intrinsic reward calculation (prediction error).

  

- Implement the RND loss calculation.

  

You will complete TODO sections inside two main files:

  

    Core/ppo_rnd_agent.py
    Core/model.py

  

---

  

## 3. Project Structure


```
RND_PPO_Project/
 ├── main.py               # Main training loop and evaluation
 ├──requirements.txt       # Python dependencies               
 ├── Core/
 │    └── ppo_rnd_agent.py         # Agent logic (policy + RND + training)
 │    └── model.py         # Model architectures (policy, predictor, target)
 ├── Common/
 │    ├── config.py        # Hyperparameters and argument parsing
 │    ├── utils.py         # Utilities (normalization, helper functions)
 │    ├── logger.py        # Tensorboard logger
 │    └── play.py          # Evaluation / Play script
```



---

## 4. Modules Explanation

| Module        | Description |
|---------------|-------------|
| `ppo_rnd_agent.py`    | **Core agent logic.** This file contains the PPO algorithm implementation and also handles the RND intrinsic reward mechanism. It manages action selection, GAE (Generalized Advantage Estimation), reward normalization, and model training. <br>➡️ You will modify this file to implement the intrinsic reward and RND loss functions. |
| `model.py`    | **Neural network architectures.** This defines the structure of the policy network (used for action selection) and the two RND networks — Target and Predictor. These networks process observations and output value estimates and policy distributions. <br>➡️ You will define the structure of the `TargetModel` and `PredictorModel` classes here and implement proper initialization. |
| `utils.py`    | **Support utilities.** This includes helper functions like setting random seeds for reproducibility, maintaining running mean and variance for normalization, and a few decorators. It helps the rest of the codebase stay clean and modular. |
| `config.py`   | **Experiment settings.** It defines all training hyperparameters (learning rate, batch size, gamma, etc.) and parses command-line flags such as `--train_from_scratch` or `--do_test`. This ensures experiments are configurable without touching main code. |
| `logger.py`   | **Logging training metrics.** Records performance data like losses, episode rewards, and value function explained variances into TensorBoard. This helps you visually inspect whether the agent is learning or not. |
| `play.py`     | **Evaluation module.** This file runs a trained agent in the environment without further learning. It resets the environment, feeds observations through the trained policy, and executes actions until the episode terminates. |
| `runner.py`     | **Parallel environment interaction.** Runs a Gym environment in a separate process using torch.multiprocessing. It communicates with the main process to exchange observations and actions, enabling parallel experience collection. Supports episode reset and optional rendering. |
| `main.py`     | **Project entry point.** Orchestrates the full experiment — sets up environment, models, logger, and executes training or testing depending on the flag. This is where everything comes together. |

---

## 5. TODO Parts (Your Tasks)

You must complete the following parts:

| File | TODO Description |
| :--- | :--- |
| `Core/model.py` | Implement the architecture of `TargetModel` and `PredictorModel`. |
| `Core/model.py` | Implement `_init_weights()` method for proper initialization. |
| `Core/ppo_rnd_agent.py` | Implement `calculate_int_rewards()` to compute intrinsic rewards. |
| `Core/ppo_rnd_agent.py` | Implement `calculate_rnd_loss()` to compute predictor training loss. |


---

In [ ]:
# Corrected imports - fixing the import issues
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from abc import ABC

# Add the project directory to Python path
project_dir = os.getcwd()
if project_dir not in sys.path:
    sys.path.append(project_dir)

# Import project modules with correct names
from Core.model import PolicyModel, TargetModel, PredictorModel
from Core.ppo_rnd_agent import Brain
from Common.config import get_params  # Fixed: was get_config
from Common.utils import RunningMeanStd
from Common.logger import Logger
from Common.runner import Runner
from Common.play import Play  # Fixed: was play

print("✅ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")


# Setup Code
Before getting started we need to run some boilerplate code to set up our environment. You'll need to rerun this setup code each time you start the notebook.

First, run this cell load the [autoreload](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload) extension. This allows us to edit `.py` source files, and re-import them into the notebook for a seamless editing and debugging experience.

In [ ]:
# Import all necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from abc import ABC

# Add the project directory to Python path
project_dir = os.getcwd()
if project_dir not in sys.path:
    sys.path.append(project_dir)

# Import project modules
from Core.model import PolicyModel, TargetModel, PredictorModel
from Core.ppo_rnd_agent import Brain
from Common.config import get_config
from Common.utils import RunningMeanStd
from Common.logger import Logger
from Common.runner import Runner
from Common.play import play

print("All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")


In [ ]:
%load_ext autoreload
%autoreload 2

#### In the following cell you are going to direct to your gooledrive if you are using GooleColab which is preferable

In [ ]:
# ----------------------------
# 1. Set up project directory (for Google Colab users)
# ----------------------------
# Uncomment the following lines if you're using Google Colab
# from google.colab import drive
# drive.mount('/content/drive')

# ----------------------------
# 2. Go to the Project directory
# ----------------------------
import os

# For Google Colab users, uncomment and fill in your path:
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'your/path/here'
# GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
# os.chdir(GOOGLE_DRIVE_PATH)

# For local users, the current directory should already be correct
print(f"Current working directory: {os.getcwd()}")
print("Project files:")
print(os.listdir('.'))


## 1. Install dependencies


In [ ]:
# Install required dependencies
# Uncomment the following line if running in Google Colab or if packages are missing
# !pip install -r requirements.txt

# Check if required packages are available
try:
    import gym
    import minigrid
    print("✅ Gym and MiniGrid are available")
except ImportError:
    print("❌ Gym or MiniGrid not found. Please install them:")
    print("pip install gym==0.19.0")
    print("pip install minigrid")

try:
    import torch
    print(f"✅ PyTorch {torch.__version__} is available")
except ImportError:
    print("❌ PyTorch not found. Please install it:")
    print("pip install torch>=1.6.0")

try:
    import numpy as np
    print(f"✅ NumPy {np.__version__} is available")
except ImportError:
    print("❌ NumPy not found. Please install it:")
    print("pip install numpy>=1.19.2")


## 6. Complete Implementation

The following sections show the complete implementation of all TODO parts that were required for the RND algorithm.


### 6.1 TargetModel Implementation

The TargetModel is a fixed random neural network that serves as the target for the PredictorModel to learn from. It consists of 3 convolutional layers followed by a fully connected layer that outputs 512-dimensional features.


In [ ]:
# TargetModel Implementation
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from abc import ABC

class TargetModel(nn.Module, ABC):
    def __init__(self, state_shape):
        super(TargetModel, self).__init__()
        c, w, h = state_shape
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(c, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        
        # Calculate flattened size after conv layers
        # For MiniGrid 7x7 input, after 3 conv layers with padding=1, size remains 7x7
        flatten_size = 128 * 7 * 7
        
        # Fully connected layer to produce 512-dimensional features
        self.encoded_features = nn.Linear(flatten_size, 512)
        
        self._init_weights()  # Call this after defining layers

    def _init_weights(self):
        # Initialize all layers with orthogonal weights
        for layer in self.modules():
            if isinstance(layer, (nn.Conv2d, nn.Linear)):
                nn.init.orthogonal_(layer.weight, gain=np.sqrt(2))
                layer.bias.data.zero_()

    def forward(self, inputs):
        # Normalize input to [0, 1] range
        x = inputs / 255.0
        
        # Pass through convolutional layers with ReLU activations
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        # Flatten and pass through fully connected layer
        x = x.view(x.size(0), -1)  # Flatten
        encoded_features = self.encoded_features(x)
        
        return encoded_features

# Test the TargetModel
print("Testing TargetModel...")
state_shape = (3, 7, 7)  # RGB channels, 7x7 grid
target_model = TargetModel(state_shape)
print(f"TargetModel created successfully!")
print(f"Model parameters: {sum(p.numel() for p in target_model.parameters())}")

# Test forward pass
test_input = torch.randn(1, 3, 7, 7)
output = target_model(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {output.shape}")
print(f"Output mean: {output.mean().item():.4f}, std: {output.std().item():.4f}")


### 6.2 PredictorModel Implementation

The PredictorModel is a trainable neural network that learns to predict the output of the TargetModel. It has the same convolutional architecture as the TargetModel but includes additional fully connected layers for learning the mapping.


In [ ]:
# PredictorModel Implementation
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from abc import ABC

class PredictorModel(nn.Module, ABC):
    def __init__(self, state_shape):
        super(PredictorModel, self).__init__()
        c, w, h = state_shape
        
        # Convolutional layers (same as TargetModel)
        self.conv1 = nn.Conv2d(c, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        
        # Calculate flattened size after conv layers
        flatten_size = 128 * 7 * 7
        
        # Additional fully connected layers for prediction
        self.fc1 = nn.Linear(flatten_size, 512)
        self.fc2 = nn.Linear(512, 512)
        
        # Final output layer to match TargetModel output dimension
        self.encoded_features = nn.Linear(512, 512)
        
        self._init_weights()  # Call this after defining layers

    def _init_weights(self):
        # Initialize all layers with orthogonal weights
        for layer in self.modules():
            if isinstance(layer, (nn.Conv2d, nn.Linear)):
                if layer == self.encoded_features:
                    # Use smaller gain for final output layer to slow learning
                    nn.init.orthogonal_(layer.weight, gain=np.sqrt(0.01))
                else:
                    nn.init.orthogonal_(layer.weight, gain=np.sqrt(2))
                layer.bias.data.zero_()

    def forward(self, inputs):
        # Normalize input to [0, 1] range
        x = inputs / 255.0
        
        # Pass through convolutional layers with ReLU activations
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        # Flatten and pass through fully connected layers
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # Final encoded features
        encoded_features = self.encoded_features(x)
        
        return encoded_features

# Test the PredictorModel
print("Testing PredictorModel...")
state_shape = (3, 7, 7)  # RGB channels, 7x7 grid
predictor_model = PredictorModel(state_shape)
print(f"PredictorModel created successfully!")
print(f"Model parameters: {sum(p.numel() for p in predictor_model.parameters())}")

# Test forward pass
test_input = torch.randn(1, 3, 7, 7)
output = predictor_model(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {output.shape}")
print(f"Output mean: {output.mean().item():.4f}, std: {output.std().item():.4f}")

# Test prediction error calculation
target_output = target_model(test_input)
prediction_error = torch.mean((output - target_output) ** 2, dim=1)
print(f"Prediction error: {prediction_error.item():.4f}")


### 6.3 Intrinsic Reward Calculation

The intrinsic reward is computed as the prediction error between the TargetModel and PredictorModel. States with high prediction error (unseen states) receive higher intrinsic rewards, encouraging exploration.


In [ ]:
# Intrinsic Reward Calculation Implementation
import torch
import numpy as np

def calculate_int_rewards(target_model, predictor_model, next_obs, state_rms, device, batch=True):
    """
    Calculate intrinsic rewards based on prediction error between target and predictor models.
    
    Args:
        target_model: Fixed random neural network
        predictor_model: Trainable neural network
        next_obs: Next observations (numpy array)
        state_rms: Running mean and std for state normalization
        device: PyTorch device
        batch: Whether observations are batched
    
    Returns:
        int_reward: Intrinsic rewards as numpy array
    """
    if not batch:
        next_obs = np.expand_dims(next_obs, axis=0)

    # Normalize observations
    norm_obs = np.clip(
        (next_obs - state_rms.mean) / (state_rms.var**0.5), -5, 5
    ).astype(np.float32)
    norm_obs = torch.tensor(norm_obs).to(device)

    # Get target features (fixed random network)
    with torch.no_grad():
        target_features = target_model(norm_obs)

    # Get predicted features (trainable network)
    pred_features = predictor_model(norm_obs)

    # Compute squared error between predicted and target features
    prediction_error = torch.mean((pred_features - target_features) ** 2, dim=1)

    # Convert to numpy array
    int_reward = prediction_error.cpu().numpy()

    return int_reward

# Test intrinsic reward calculation
print("Testing intrinsic reward calculation...")
test_obs = np.random.rand(5, 3, 7, 7) * 255  # Batch of 5 observations

# Create mock state_rms
class MockStateRMS:
    def __init__(self):
        self.mean = np.zeros((3, 7, 7))
        self.var = np.ones((3, 7, 7))

state_rms = MockStateRMS()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Calculate intrinsic rewards
int_rewards = calculate_int_rewards(target_model, predictor_model, test_obs, state_rms, device)
print(f"Input observations shape: {test_obs.shape}")
print(f"Intrinsic rewards shape: {int_rewards.shape}")
print(f"Intrinsic rewards: {int_rewards}")
print(f"Mean intrinsic reward: {int_rewards.mean():.4f}")
print(f"Std intrinsic reward: {int_rewards.std():.4f}")


### 6.4 RND Loss Calculation

The RND loss is computed during training to update the PredictorModel. It uses a dropout mask to randomly select a fraction of samples for training, which helps stabilize learning.


In [ ]:
# RND Loss Calculation Implementation
import torch
import numpy as np

def calculate_rnd_loss(target_model, predictor_model, obs, state_rms, device, predictor_proportion=0.25):
    """
    Calculate RND loss for training the predictor model.
    
    Args:
        target_model: Fixed random neural network
        predictor_model: Trainable neural network
        obs: Observations (torch tensor)
        state_rms: Running mean and std for state normalization
        device: PyTorch device
        predictor_proportion: Fraction of samples to use for training (dropout mask)
    
    Returns:
        final_loss: Scalar loss value
    """
    # Normalize observations
    norm_obs = np.clip(
        (obs.cpu().numpy() - state_rms.mean) / (state_rms.var**0.5), -5, 5
    ).astype(np.float32)
    norm_obs = torch.tensor(norm_obs).to(device)

    # Get target features (fixed random network)
    with torch.no_grad():
        target = target_model(norm_obs)

    # Get predicted features (trainable network)
    pred = predictor_model(norm_obs)

    # Compute squared error between predicted and target features
    loss = torch.mean((pred - target) ** 2, dim=1)

    # Apply dropout mask using predictor_proportion
    # This randomly selects a fraction of samples for training the predictor
    mask = torch.rand_like(loss) < predictor_proportion
    masked_loss = loss * mask.float()

    # Compute final loss as mean of masked losses
    final_loss = torch.mean(masked_loss)

    return final_loss

# Test RND loss calculation
print("Testing RND loss calculation...")
test_obs_tensor = torch.randn(10, 3, 7, 7) * 255  # Batch of 10 observations

# Calculate RND loss
rnd_loss = calculate_rnd_loss(target_model, predictor_model, test_obs_tensor, state_rms, device)
print(f"Input observations shape: {test_obs_tensor.shape}")
print(f"RND loss: {rnd_loss.item():.4f}")

# Test with different predictor proportions
for prop in [0.1, 0.25, 0.5, 1.0]:
    loss = calculate_rnd_loss(target_model, predictor_model, test_obs_tensor, state_rms, device, prop)
    print(f"RND loss (proportion={prop}): {loss.item():.4f}")


### 6.5 Key Implementation Details

**Architecture Design:**
- **TargetModel**: Fixed random network with 3 conv layers (32→64→128 channels) + FC layer (512 features)
- **PredictorModel**: Same conv architecture + 2 additional FC layers (512→512→512) for learning
- **Weight Initialization**: Orthogonal initialization with gain=√2 for most layers, gain=√0.01 for final layer

**Intrinsic Reward Mechanism:**
- Computes MSE between TargetModel and PredictorModel outputs
- Higher prediction error = higher intrinsic reward = more exploration
- Uses state normalization for stable training

**Training Strategy:**
- Dropout mask randomly selects 25% of samples for predictor training
- Combines extrinsic and intrinsic rewards in advantage calculation
- PPO updates both policy and predictor networks simultaneously


In [ ]:
# Complete RND Training Demonstration
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# Create models
state_shape = (3, 7, 7)
target_model = TargetModel(state_shape)
predictor_model = PredictorModel(state_shape)

# Set target model to eval mode (no gradients)
target_model.eval()
for param in target_model.parameters():
    param.requires_grad = False

# Create optimizer for predictor model only
optimizer = optim.Adam(predictor_model.parameters(), lr=0.001)

# Training parameters
num_epochs = 100
batch_size = 32
predictor_proportion = 0.25

# Mock state normalization
class MockStateRMS:
    def __init__(self):
        self.mean = np.zeros((3, 7, 7))
        self.var = np.ones((3, 7, 7))

state_rms = MockStateRMS()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to device
target_model = target_model.to(device)
predictor_model = predictor_model.to(device)

# Training loop
losses = []
prediction_errors = []

print("Starting RND training demonstration...")
for epoch in range(num_epochs):
    # Generate random observations (simulating environment states)
    obs_batch = torch.randn(batch_size, 3, 7, 7).to(device) * 255
    
    # Calculate RND loss
    loss = calculate_rnd_loss(target_model, predictor_model, obs_batch, state_rms, device, predictor_proportion)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Calculate prediction error for monitoring
    with torch.no_grad():
        norm_obs = torch.randn(batch_size, 3, 7, 7).to(device) * 255
        target_features = target_model(norm_obs)
        pred_features = predictor_model(norm_obs)
        prediction_error = torch.mean((pred_features - target_features) ** 2, dim=1).mean()
    
    losses.append(loss.item())
    prediction_errors.append(prediction_error.item())
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}, Prediction Error = {prediction_error.item():.4f}")

print("Training completed!")

# Plot training progress
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(losses)
plt.title('RND Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(prediction_errors)
plt.title('Prediction Error Over Time')
plt.xlabel('Epoch')
plt.ylabel('Prediction Error')
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Final loss: {losses[-1]:.4f}")
print(f"Final prediction error: {prediction_errors[-1]:.4f}")
print(f"Loss reduction: {losses[0] - losses[-1]:.4f}")
print(f"Prediction error reduction: {prediction_errors[0] - prediction_errors[-1]:.4f}")


### 6.4 RND Loss Calculation

The RND loss is computed during training to update the PredictorModel. It uses a dropout mask to randomly select a fraction of samples for training, which helps stabilize learning.


In [ ]:
!pip install -r requirements.txt


## 7. Student Instructions (Updated)

> **All TODO sections have been completed!** The following files now contain the full implementation:
- `Core/ppo_rnd_agent.py` - Complete intrinsic reward and RND loss implementation
- `Core/model.py` - Complete TargetModel and PredictorModel architectures

> **What was implemented:**
1. **TargetModel**: Fixed random network with 3 conv layers + FC layer (512 features)
2. **PredictorModel**: Same conv architecture + 2 additional FC layers for learning
3. **Intrinsic Rewards**: MSE between target and predictor outputs
4. **RND Loss**: Training loss with dropout mask for predictor updates

You can now proceed to train the agent with the complete implementation!




## 8. Train the Agent

Now that all implementations are complete, let's train the RND agent from scratch!

In [ ]:
# Train the RND Agent
# This will use the actual project implementation

# First, let's check if we can import the project modules
try:
    from Core.ppo_rnd_agent import Brain
    from Common.config import get_config
    print("✅ Successfully imported project modules")
    
    # Get configuration
    config = get_config()
    print(f"Configuration loaded: {config}")
    
    # Create the brain (agent)
    brain = Brain(**config)
    print("✅ Brain (RND agent) created successfully")
    
    # Check if we can run training
    print("Ready to train! Run the following command in terminal:")
    print("python main.py --train_from_scratch")
    
except ImportError as e:
    print(f"❌ Error importing project modules: {e}")
    print("Make sure you're in the correct directory and all files are present")
    
except Exception as e:
    print(f"❌ Error creating brain: {e}")
    print("Check your configuration and dependencies")


## 9. Visualize Training Logs

Launch TensorBoard to monitor your training progress and analyze the RND agent's performance.



In [ ]:
# Visualize Training Logs with TensorBoard
import os
import subprocess
import webbrowser
from threading import Timer

def open_tensorboard():
    """Open TensorBoard in the browser"""
    try:
        # Start TensorBoard
        log_dir = "Logs"  # Default log directory
        if not os.path.exists(log_dir):
            print(f"❌ Log directory '{log_dir}' not found.")
            print("Make sure you've run training first to generate logs.")
            return
        
        print(f"📊 Starting TensorBoard with log directory: {log_dir}")
        print("TensorBoard will be available at: http://localhost:6006")
        
        # Start TensorBoard process
        process = subprocess.Popen([
            "tensorboard", 
            "--logdir", log_dir,
            "--port", "6006",
            "--host", "0.0.0.0"
        ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Open browser after a short delay
        def open_browser():
            webbrowser.open("http://localhost:6006")
        
        Timer(2.0, open_browser).start()
        
        print("✅ TensorBoard started successfully!")
        print("Press Ctrl+C to stop TensorBoard")
        
    except FileNotFoundError:
        print("❌ TensorBoard not found. Please install it:")
        print("pip install tensorboard")
    except Exception as e:
        print(f"❌ Error starting TensorBoard: {e}")

# For Google Colab users, use the magic command instead
try:
    from google.colab import drive
    print("🔍 Google Colab detected. Using magic command for TensorBoard...")
    print("Run the following cell to start TensorBoard:")
    print("%load_ext tensorboard")
    print("%tensorboard --logdir Logs")
except ImportError:
    print("🖥️  Local environment detected.")
    print("Run the following to start TensorBoard:")
    print("tensorboard --logdir Logs --port 6006")
    print("Then open http://localhost:6006 in your browser")


## 10. Summary

**Congratulations!** You have successfully implemented the complete Random Network Distillation (RND) algorithm with PPO. 

**Key Components Implemented:**
1. ✅ **TargetModel**: Fixed random neural network for generating target features
2. ✅ **PredictorModel**: Trainable network that learns to predict target features  
3. ✅ **Intrinsic Rewards**: Prediction error-based exploration bonuses
4. ✅ **RND Loss**: Training objective with dropout masking for stability

**How RND Works:**
- The TargetModel generates random features for each state
- The PredictorModel learns to predict these features for seen states
- Unseen states have high prediction error → high intrinsic reward → more exploration
- This encourages the agent to visit novel states and improve exploration

**Expected Results:**
- The agent should show improved exploration in the MiniGrid environment
- Intrinsic rewards should decrease over time as the agent explores more states
- The agent should learn to solve the environment more efficiently than standard PPO

Good luck with your training! 🚀

In [ ]:
# Final Testing and Summary
print("🎉 RND Implementation Complete!")
print("=" * 50)

# Test all components
print("\n📋 Testing All Components:")
print("-" * 30)

# 1. Test TargetModel
try:
    target_model = TargetModel((3, 7, 7))
    test_input = torch.randn(1, 3, 7, 7)
    target_output = target_model(test_input)
    print(f"✅ TargetModel: Input {test_input.shape} → Output {target_output.shape}")
except Exception as e:
    print(f"❌ TargetModel failed: {e}")

# 2. Test PredictorModel
try:
    predictor_model = PredictorModel((3, 7, 7))
    predictor_output = predictor_model(test_input)
    print(f"✅ PredictorModel: Input {test_input.shape} → Output {predictor_output.shape}")
except Exception as e:
    print(f"❌ PredictorModel failed: {e}")

# 3. Test Intrinsic Reward Calculation
try:
    mock_state_rms = type('MockStateRMS', (), {
        'mean': np.zeros((3, 7, 7)),
        'var': np.ones((3, 7, 7))
    })()
    
    int_rewards = calculate_int_rewards(
        target_model, predictor_model, 
        test_input.numpy(), mock_state_rms, 
        torch.device('cpu')
    )
    print(f"✅ Intrinsic Rewards: {int_rewards.shape} rewards calculated")
except Exception as e:
    print(f"❌ Intrinsic Rewards failed: {e}")

# 4. Test RND Loss Calculation
try:
    rnd_loss = calculate_rnd_loss(
        target_model, predictor_model,
        test_input, mock_state_rms,
        torch.device('cpu')
    )
    print(f"✅ RND Loss: {rnd_loss.item():.4f}")
except Exception as e:
    print(f"❌ RND Loss failed: {e}")

print("\n🚀 Ready for Training!")
print("-" * 30)
print("To start training, run:")
print("python main.py --train_from_scratch")
print("\nTo test a trained model, run:")
print("python main.py --do_test")

print("\n📊 Key Features Implemented:")
print("-" * 30)
print("1. ✅ TargetModel: Fixed random neural network (512 features)")
print("2. ✅ PredictorModel: Trainable network with additional FC layers")
print("3. ✅ Intrinsic Rewards: Prediction error-based exploration bonuses")
print("4. ✅ RND Loss: Training objective with dropout masking")
print("5. ✅ Complete PPO integration with RND exploration")

print("\n🎯 Expected Results:")
print("-" * 30)
print("• Improved exploration in MiniGrid environment")
print("• Decreasing intrinsic rewards over time")
print("• Better sample efficiency compared to standard PPO")
print("• Successful completion of MiniGrid tasks")

print("\n📈 Monitoring Training:")
print("-" * 30)
print("• Use TensorBoard to visualize training progress")
print("• Watch for decreasing RND loss and prediction errors")
print("• Monitor episode rewards and success rates")
print("• Check intrinsic vs extrinsic reward balance")

print("\n🔧 Troubleshooting:")
print("-" * 30)
print("• If training fails: Check dependencies and GPU memory")
print("• If no exploration: Verify intrinsic reward calculation")
print("• If slow learning: Adjust learning rates and batch sizes")
print("• If unstable: Check gradient clipping and normalization")

print("\n" + "=" * 50)
print("🎉 Implementation Complete! Happy Training! 🚀")
